# Use Tavily as a tool to search the web
 Make sure you have your tavily API key in your .env file

In [1]:
from typing import Any

from beeai_framework.agents.react.agent import ReActAgent
from beeai_framework.agents.react.types import ReActAgentRunOutput
from beeai_framework.backend.chat import ChatModel
from beeai_framework.emitter.emitter import Emitter, EventMeta
from beeai_framework.emitter.types import EmitterOptions
from beeai_framework.memory.unconstrained_memory import UnconstrainedMemory
from dotenv import load_dotenv
import os
from beeai_framework.tools import Tool, tool
from langchain_community.tools import TavilySearchResults  # Import LangChain Tavily tool
from typing import Any
import getpass
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper





In [2]:
from langchain_core.globals import set_debug

set_debug(False)  # Disable debug mode


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ.get("TAVILY_API_KEY")


'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'

In [4]:
#create the functions needed to run a re-act agent that can call tools

async def process_agent_events(event_data: dict[str, Any], event_meta: EventMeta) -> None:
    """Process agent events and log appropriately"""

    if event_meta.name == "error":
        print("Agent 🤖 : ", event_data["error"])
    elif event_meta.name == "retry":
        print("Agent 🤖 : ", "retrying the action...")
    elif event_meta.name == "update":
        print(f"Agent({event_data['update']['key']}) 🤖 : ", event_data["update"]["parsedValue"])


# Observe the agent
async def observer(emitter: Emitter) -> None:
    emitter.on("*.*", process_agent_events, EmitterOptions(match_nested=True))

In [5]:
#construct a chat model using ollama. Make sure you have ollama running with the correct model.
chat_model: ChatModel = ChatModel.from_name("ollama:granite3.1-dense:2b")

In [6]:

# Define the tool using the `tool` decorator
@tool
def tavily_search_tool(query: str) -> str:
    """
    Perform a web search for the latest and most relevant information available online.
    
    Args:
        query (str): The search query to look up on the web which comes from the websearch_query.
    
    Returns:
        str: The top search results based on the Tavily API.
    """
    if not isinstance(query, str) or not query.strip():
        raise ValueError("Invalid query input. Expected a non-empty string.")

    tavily_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=False)

    searchresults = tavily_tool.invoke({"query": query})
    return searchresults



In [7]:

# Construct Re-Agent instance with the chat model
agent = ReActAgent(llm=chat_model, tools=[tavily_search_tool], memory=UnconstrainedMemory())
#run the re-act agent enabled with tavily search tool 
result: ReActAgentRunOutput = await agent.run("What is the population of California?").observe(observer)

Agent 🤖 :  The generated output does not adhere to the schema.
Transition from 'thought' to 'tool_input' does not exist!
Agent 🤖 :  The generated output does not adhere to the schema.
Transition from 'thought' to 'tool_input' does not exist!
Agent 🤖 :  The generated output does not adhere to the schema.
Transition from 'thought' to 'tool_input' does not exist!
Agent 🤖 :  retrying the action...
Agent(thought) 🤖 :  I need to perform a web search to find out the current population of California.
Agent(tool_name) 🤖 :  tavily_search_tool
Agent(tool_input) 🤖 :  {'query': 'California population'}
Agent(tool_output) 🤖 :  [{'title': "California's Population is Increasing", 'url': 'https://www.gov.ca.gov/2024/04/30/californias-population-is-increasing/', 'content': "SACRAMENTO – California's population increased by 67,000 people last year to 39,128,162, according to new data released today by the California", 'score': 0.90810597, 'raw_content': "Published Time: 2024-04-30T16:10:29+00:00\nCalifor

Alternatively you can use langchain's API Wrapper

In [ ]:
def tavily_search_tool(query: str) -> str:
    """
    Perform a web search for the latest and most relevant information available online.
    
    Args:
        query (str): The search query to look up on the web which comes from the websearch_query.
    
    Returns:
        str: The top search results based on the Tavily API.
    """
    if not isinstance(query, str) or not query.strip():
        raise ValueError("Invalid query input. Expected a non-empty string.")

    search = TavilySearchAPIWrapper()
    tavily_tool = TavilySearchResults(api_wrapper=search)
    return tavily_tool.run(query)



tavily_search_tool("what is the population of california?")